In [25]:
import pandas as pd
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

sns.set_theme(style="ticks")

pd.set_option("display.max_columns", None) 
#pd.set_option("display.max_rows", None) 

In [26]:
df_baseline_rank = pd.read_csv("../baseline_rank_trace_2.csv")
df_gk_rank = pd.read_csv("../baseline_rank_trace.csv")
df_qdig_capacity_rank = pd.read_csv("../baseline_rank_trace.csv")
df_qdig_update_rank = pd.read_csv("../baseline_rank_trace.csv")
df_qdig_weight_rank = pd.read_csv("../baseline_rank_trace.csv")
#df_baseline_quant = pd.read_csv("../baseline_quant_trace.csv")
#df_gk_quant = pd.read_csv("../baseline_quant_trace.csv")
#df_qdig_capacity_quant = pd.read_csv("../baseline_quant_trace.csv")
#df_qdig_update_quant = pd.read_csv("../baseline_quant_trace.csv")
#df_qdig_weight_quant = pd.read_csv("../baseline_quant_trace.csv")

In [34]:
df_gk_rank.columns

Index(['id', 'val', 'epsilon', 'univ', 'query', 'time_update', 'time_query',
       'mem_usage', 'comp_count', 'comp_mem_avg', 'value'],
      dtype='object')

In [28]:
TARGETS = df_baseline_rank["val"].unique()
TARGETS.sort()
TARGETS

array([0, 3, 4, 5, 6, 7])

In [29]:
METRICAS = ["value", "time_query", "time_update", "mem_usage", "compress_count", "compress_saved_mem_avg"]

In [30]:
df_baseline_rank = df_baseline_rank[df_baseline_rank["val"].isin(TARGETS)]
df_baseline_rank

,id,val,epsilon,univ,query,time_update,time_query,mem_usage,comp_count,comp_mem_avg,value,weight
0,1,0,0.01,100000,12499,990287,14625,2114328,0,0,29808,264288
1,6,5,0.01,10000,9999,982489,132890,28603320,0,0,3575410,3575412
2,7,7,0.05,100000,49999,1101192,297737,27368832,0,0,3355711,3421101
3,4,4,0.10,100000,16666,1308820,168064,28617720,0,0,3576197,3577212
4,3,4,0.20,10000,2499,1248806,342341,28602776,0,0,3566981,3575344
...,...,...,...,...,...,...,...,...,...,...,...,...
3595,7,7,0.10,1000000,499999,4454758,195435,28248248,0,0,3505738,3531028
3596,9,6,0.20,1000000,249999,4644701,168812,28412664,0,0,3531027,3551580
3597,1,7,0.20,1000000,499999,4840951,160704,28248248,0,0,3505738,3531028
3598,7,5,0.05,10000,1249,4814072,117171,28603320,0,0,3547772,3575412


In [39]:
tgt_univ = df_gk_rank.groupby(['val', 'univ', 'query']).agg({'value': ['min'], 'time_update': ['mean'], 'time_query': ['mean'], 'mem_usage': ['mean']})
tgt_univ.columns = ['value', 'time_update', 'time_query', 'mem_usage']
tgt_univ = tgt_univ.reset_index()
print(tgt_univ)

    val     univ   query    value  time_update  time_query   mem_usage
0     0    10000    1249     3021  4362993.450    1961.075    192152.0
1     0    10000    1666     3957  4476006.525    2646.200    192152.0
2     0    10000    2499     5916  4411487.825    7258.025    192152.0
3     0    10000    4999    12041  4607978.400    4090.725    192152.0
4     0    10000    9999    24013  4330893.500    3105.500    192152.0
..  ...      ...     ...      ...          ...         ...         ...
85    7  1000000  124999  3435561  4330631.850  782193.650  28248248.0
86    7  1000000  166666  3452916  4554645.225  788337.075  28248248.0
87    7  1000000  249999  3474855  4415831.675  783458.600  28248248.0
88    7  1000000  499999  3505738  4423205.525  798484.275  28248248.0
89    7  1000000  999999  3531028  4481367.750  815940.050  28248248.0

[90 rows x 7 columns]


In [32]:
tgt_univ = df_qdig_weight_rank.groupby(['val', 'univ', 'query']).agg({'value': ['min']})
tgt_univ.columns = ['value']
tgt_univ = tgt_univ.reset_index()
print(tgt_univ)

     val     univ   query  epsilon    value
0      0    10000    1249     0.01     3021
1      0    10000    1249     0.05     3021
2      0    10000    1249     0.10     3021
3      0    10000    1249     0.20     3021
4      0    10000    1666     0.01     3957
..   ...      ...     ...      ...      ...
355    7  1000000  499999     0.20  3505738
356    7  1000000  999999     0.01  3531028
357    7  1000000  999999     0.05  3531028
358    7  1000000  999999     0.10  3531028
359    7  1000000  999999     0.20  3531028

[360 rows x 5 columns]


In [38]:
tgt_univ = df_baseline_rank.groupby(['val', 'univ', 'query']).agg({'weight': ['mean']})
error = df_baseline_rank.groupby(['val', 'univ', 'query']).agg({'epsilon': ['mean']})
tgt_univ['error_allowed'] = error * tgt_univ['weight'];
tgt_univ.columns = ['weight', 'error_allowed'];
tgt_univ = tgt_univ.reset_index()
print(tgt_univ)

ValueError: cannot join with no overlapping index names